In [ ]:
import os
import re


def parse_and_save_final(input_file_path):
    """
    (최종 개선안) 먼저 문서를 나눈 뒤, 각 덩어리를 개별적으로 파싱하여 안정성을 극대화합니다.
    """
    # 1. 결과를 저장할 폴더 생성 (이전 시도와 겹치지 않게 새 이름 사용)
    output_folder = "output_from_txt_final"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 2. 텍스트 파일 읽기
    try:
        with open(input_file_path, "r", encoding="utf-8") as f:
            full_text = f.read()
    except FileNotFoundError:
        print(f"❌ 오류: '{input_file_path}' 파일을 찾을 수 없습니다.")
        return

    # 3. (1단계) '- id:' 패턴을 기준으로 전체 텍스트를 문서별로 분할
    # (?=...)는 '긍정형 전방 탐색'으로, 분리자로 사용된 '- id:' 패턴을 지우지 않고 다음 덩어리에 포함시킵니다.
    document_chunks = re.split(r"(?=\s*-\s*id:)", full_text)

    # 분할 결과 첫 번째 요소는 비어있을 수 있으므로, 실제 내용이 있는 덩어리만 필터링합니다.
    valid_chunks = [chunk for chunk in document_chunks if chunk.strip()]

    if len(valid_chunks) == 0:
        print("⚠️ 문서 데이터를 찾을 수 없습니다. 파일 내용을 확인해주세요.")
        return

    print(
        f"총 {len(valid_chunks)}개의 문서 블록을 분리했습니다. 내용 추출을 시작합니다..."
    )

    successful_saves = 0
    # 4. (2단계) 나누어진 각 덩어리를 개별적으로 분석
    for i, chunk in enumerate(valid_chunks):
        try:
            # 이 덩어리 안에서 ID를 찾습니다.
            id_match = re.search(r"-\s*id:\s*([a-f0-9\-]+)", chunk)

            # 이 덩어리 안에서 page_content를 찾습니다.
            content_match = re.search(
                r"page_content:\s*\|-(.*?)type:\s*Document", chunk, re.DOTALL
            )

            if id_match and content_match:
                doc_id = id_match.group(1).strip()
                page_content = content_match.group(1).strip()

                # 파일로 저장
                output_file_path = os.path.join(output_folder, f"{doc_id}.txt")
                with open(output_file_path, "w", encoding="utf-8") as f_out:
                    f_out.write(page_content)

                print(f"✅ 성공: #{i+1} '{output_file_path}' 저장 완료")
                successful_saves += 1
            else:
                # 어떤 덩어리가 실패했는지 알려주어 디버깅을 돕습니다.
                print(
                    f"⚠️ 경고: #{i+1}번째 문서 블록에서 ID 또는 콘텐츠 패턴을 찾지 못했습니다. 건너뜁니다."
                )

        except Exception as e:
            print(f"❌ 오류: #{i+1}번째 문서 블록 처리 중 예외 발생 - {e}")

    print(
        f"\n🎉 모든 작업 완료! 총 {len(valid_chunks)}개 중 {successful_saves}개 파일 저장 성공."
    )
    print(f"'{output_folder}' 폴더를 확인하세요.")

In [6]:
file_to_parse = "/Users/limseongbeom/Desktop/Code/VOLTAI/evaluation/per_model_test/pl/CONTEXT_gemini-1.5-pro/paper_11.txt"

parse_and_save_final(file_to_parse)

총 10개의 문서 블록을 분리했습니다. 내용 추출을 시작합니다...
⚠️ 경고: #1번째 문서 블록에서 ID 또는 콘텐츠 패턴을 찾지 못했습니다. 건너뜁니다.
✅ 성공: #2 'output_from_txt_final/e3e3f314-0bfb-4702-b6e6-0746bd142a92.txt' 저장 완료
⚠️ 경고: #3번째 문서 블록에서 ID 또는 콘텐츠 패턴을 찾지 못했습니다. 건너뜁니다.
✅ 성공: #4 'output_from_txt_final/99a60b0e-5fa1-4125-98f8-15e8889da1f5.txt' 저장 완료
✅ 성공: #5 'output_from_txt_final/9e241b25-c034-427a-8438-6e95309b04c6.txt' 저장 완료
⚠️ 경고: #6번째 문서 블록에서 ID 또는 콘텐츠 패턴을 찾지 못했습니다. 건너뜁니다.
⚠️ 경고: #7번째 문서 블록에서 ID 또는 콘텐츠 패턴을 찾지 못했습니다. 건너뜁니다.
⚠️ 경고: #8번째 문서 블록에서 ID 또는 콘텐츠 패턴을 찾지 못했습니다. 건너뜁니다.
⚠️ 경고: #9번째 문서 블록에서 ID 또는 콘텐츠 패턴을 찾지 못했습니다. 건너뜁니다.
✅ 성공: #10 'output_from_txt_final/373be9fa-4c23-4100-b891-925bb439e8ee.txt' 저장 완료

🎉 모든 작업 완료! 총 10개 중 4개 파일 저장 성공.
'output_from_txt_final' 폴더를 확인하세요.


In [ ]:
import yaml

# YAML 파일 경로


# YAML 읽기
with open(file_to_parse, "r", encoding="utf-8") as f:
    data = yaml.safe_load(f)

# 확인
print(len(data["documents"]))  # 문서 수 확인

9


In [ ]:
documents = data["documents"]

for i, doc in enumerate(documents):
    content = doc["page_content"]
    doc_id = doc["id"]
    print(f"문서 {i+1}: {doc_id}, 글자 수: {len(content)}")

문서 1: e3e3f314-0bfb-4702-b6e6-0746bd142a92, 글자 수: 3203
문서 2: a9775846-e669-49ab-a0bd-45da88ea3d4a, 글자 수: 2959
문서 3: 99a60b0e-5fa1-4125-98f8-15e8889da1f5, 글자 수: 3573
문서 4: 9e241b25-c034-427a-8438-6e95309b04c6, 글자 수: 2851
문서 5: 32eda8a6-1672-4da2-8d5f-293b8d8d80dc, 글자 수: 2886
문서 6: f3a372fe-73a3-4837-b5c5-e899894f676c, 글자 수: 2008
문서 7: 130fbcda-2ea8-4a64-b571-47081b9af8d2, 글자 수: 3183
문서 8: 7d7aa5cd-ab8b-4a53-9bca-fb4c0c1e1fdb, 글자 수: 3213
문서 9: 373be9fa-4c23-4100-b891-925bb439e8ee, 글자 수: 3351


In [ ]:
import os

output_dir = "context\output_docs_gemini_1.5_pro_11"
os.makedirs(output_dir, exist_ok=True)

for i, doc in enumerate(documents):
    content = doc["page_content"]
    doc_id = doc["id"]
    file_path = os.path.join(output_dir, f"{i+1}_{doc_id}.txt")

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content)

In [ ]:
import os
import yaml

# 1️⃣ 기본 경로 설정
base_dir = "/Users/limseongbeom/Desktop/Code/VOLTAI/evaluation/per_model_test/pl"

# 2️⃣ 새 출력 폴더 경로
output_base_dir = (
    "/Users/limseongbeom/Desktop/Code/VOLTAI/evaluation/per_model_test/parsed_docs"
)

# 3️⃣ 모델 리스트
models = ["gemini-1.5-pro", "gemini-2.0-flash", "gpt-4o", "gpt-4o-mini"]

# 4️⃣ 모델별 폴더 반복
for model in models:
    model_folder = os.path.join(base_dir, f"CONTEXT_{model}")
    if not os.path.exists(model_folder):
        print(f"폴더 없음: {model_folder}")
        continue

    # 5️⃣ 모델별 폴더 내 파일 반복
    for file_name in os.listdir(model_folder):
        if not file_name.endswith(".txt"):
            continue  # txt 파일만 처리

        file_to_parse = os.path.join(model_folder, file_name)

        # 6️⃣ YAML 읽기
        with open(file_to_parse, "r", encoding="utf-8") as f:
            data = yaml.safe_load(f)

        documents = data.get("documents", [])
        print(f"{file_name} → 문서 수: {len(documents)}")

        # 7️⃣ 출력 폴더 설정 (모델명 + 파일명 기준)
        file_base_name = os.path.splitext(file_name)[0]
        output_dir = os.path.join(output_base_dir, f"{model}_{file_base_name}")
        os.makedirs(output_dir, exist_ok=True)

        # 8️⃣ 문서별 개별 txt 저장
        for i, doc in enumerate(documents):
            content = doc["page_content"]
            doc_id = doc["id"]
            output_file_path = os.path.join(output_dir, f"{i+1}_{doc_id}.txt")
            with open(output_file_path, "w", encoding="utf-8") as f:
                f.write(content)

        print(f"{file_name} 완료 → {len(documents)}개 문서 저장됨")

paper_16.txt → 문서 수: 9
paper_16.txt 완료 → 9개 문서 저장됨
paper_11.txt → 문서 수: 9
paper_11.txt 완료 → 9개 문서 저장됨
paper_39.txt → 문서 수: 9
paper_39.txt 완료 → 9개 문서 저장됨
paper_22.txt → 문서 수: 6
paper_22.txt 완료 → 6개 문서 저장됨
paper_35.txt → 문서 수: 9
paper_35.txt 완료 → 9개 문서 저장됨
paper_40.txt → 문서 수: 9
paper_40.txt 완료 → 9개 문서 저장됨
paper_41.txt → 문서 수: 8
paper_41.txt 완료 → 8개 문서 저장됨
paper_42.txt → 문서 수: 9
paper_42.txt 완료 → 9개 문서 저장됨
paper_56.txt → 문서 수: 9
paper_56.txt 완료 → 9개 문서 저장됨
paper_44.txt → 문서 수: 9
paper_44.txt 완료 → 9개 문서 저장됨
paper_16.txt → 문서 수: 9
paper_16.txt 완료 → 9개 문서 저장됨
paper_11.txt → 문서 수: 9
paper_11.txt 완료 → 9개 문서 저장됨
paper_39.txt → 문서 수: 9
paper_39.txt 완료 → 9개 문서 저장됨
paper_22.txt → 문서 수: 6
paper_22.txt 완료 → 6개 문서 저장됨
paper_35.txt → 문서 수: 9
paper_35.txt 완료 → 9개 문서 저장됨
paper_40.txt → 문서 수: 9
paper_40.txt 완료 → 9개 문서 저장됨
paper_41.txt → 문서 수: 8
paper_41.txt 완료 → 8개 문서 저장됨
paper_42.txt → 문서 수: 9
paper_42.txt 완료 → 9개 문서 저장됨
paper_56.txt → 문서 수: 9
paper_56.txt 완료 → 9개 문서 저장됨
paper_44.txt → 문서 수: 9
paper_44

In [ ]:
import os
import pandas as pd

# 1️⃣ 경로 설정
parsed_docs_dir = (
    "/Users/limseongbeom/Desktop/Code/VOLTAI/evaluation/per_model_test/parsed_docs"
)
json_base_dir = "/Users/limseongbeom/Desktop/Code/VOLTAI/evaluation/per_model_test/pl"

# 2️⃣ 모델 리스트
models = ["gemini-1.5-pro", "gemini-2.0-flash", "gpt-4o", "gpt-4o-mini"]

# 3️⃣ DataFrame 저장용 리스트
records = []

for model in models:
    model_json_folder = os.path.join(json_base_dir, model)
    if not os.path.exists(model_json_folder):
        print(f"폴더 없음: {model_json_folder}")
        continue

    for paper_folder_name in os.listdir(model_json_folder):
        paper_json_folder = os.path.join(model_json_folder, paper_folder_name)
        if not os.path.isdir(paper_json_folder):
            continue

        # JSON 파일 읽기 (폴더 안에 하나만 존재한다고 가정)
        json_files = [f for f in os.listdir(paper_json_folder) if f.endswith(".json")]
        if not json_files:
            print(f"JSON 없음: {paper_json_folder}")
            continue
        json_path = os.path.join(paper_json_folder, json_files[0])
        with open(json_path, "r", encoding="utf-8") as f:
            json_str = f.read()

        # TXT 파일 읽기
        parsed_paper_folder_name = f"{model}_{paper_folder_name.replace('_output','')}"
        parsed_paper_folder = os.path.join(parsed_docs_dir, parsed_paper_folder_name)
        context_list = []

        if os.path.exists(parsed_paper_folder):
            for txt_file in sorted(os.listdir(parsed_paper_folder)):
                if txt_file.endswith(".txt"):
                    txt_path = os.path.join(parsed_paper_folder, txt_file)
                    with open(txt_path, "r", encoding="utf-8") as f:
                        context_list.append(f.read())
        else:
            print(f"파싱 문서 폴더 없음: {parsed_paper_folder}")

        # 레코드 저장 (model 열 추가)
        paper_name = paper_folder_name.replace("_output", "")
        records.append(
            {
                "model": model,  # ⭐ 여기 추가
                "paper": paper_name,
                "output": json_str,
                "context": context_list,
            }
        )

# 4️⃣ DataFrame 생성
df = pd.DataFrame(records)
print(df.head())

            model      paper  \
0  gemini-1.5-pro  paper_016   
1  gemini-1.5-pro  paper_022   
2  gemini-1.5-pro  paper_041   
3  gemini-1.5-pro  paper_011   
4  gemini-1.5-pro  paper_056   

                                              output  \
0  {\n    "Particle size": {\n        "NCM": "50 ...   
1  {\n    "Particle size": {\n        "pristine":...   
2  {\n    "Particle size": {\n        "pristine N...   
3  {\n    "Particle size": {\n        "NR0": "200...   
4  {\n    "Particle size": {\n        "pristine N...   

                                             context  
0  [Figure 2 shows the SEM and TEM images of NCM ...  
1  [Results and Discussion\nThe XRD patterns of p...  
2  [The SEM images for the pristine NCM-811, g-C3...  
3  [6\n\nA. Habibi et al. / Journal of Alloys and...  
4  [For confirming the microstructural informatio...  


In [2]:
df

,model,paper,output,context
0,gemini-1.5-pro,paper_016,"{\n ""Particle size"": {\n ""NCM"": ""50 ...",[Figure 2 shows the SEM and TEM images of NCM ...
1,gemini-1.5-pro,paper_022,"{\n ""Particle size"": {\n ""pristine"":...",[Results and Discussion\nThe XRD patterns of p...
2,gemini-1.5-pro,paper_041,"{\n ""Particle size"": {\n ""pristine N...","[The SEM images for the pristine NCM-811, g-C3..."
3,gemini-1.5-pro,paper_011,"{\n ""Particle size"": {\n ""NR0"": ""200...",[6\n\nA. Habibi et al. / Journal of Alloys and...
4,gemini-1.5-pro,paper_056,"{\n ""Particle size"": {\n ""pristine N...",[For confirming the microstructural informatio...
5,gemini-1.5-pro,paper_035,"{\n ""Particle size"": {\n ""S1bare"": ""...","[4. Conclusions\n\nand\n\ncurrent\n\n(GITT),\n..."
6,gemini-1.5-pro,paper_044,"{\n ""Particle size"": {\n ""NCM622"": n...",[Figure 1a. NCM622 particles were first soaked...
7,gemini-1.5-pro,paper_039,"{\n ""Particle size"": {\n ""N92"": ""gra...",[2205122 (2 of 13)\n\n© 2022 Wiley-VCH GmbH\n...
8,gemini-1.5-pro,paper_042,"{\n ""Particle size"": {\n ""N-NCM"": ""1...",[3. Results and discussion\n\nFig. 2 shows the...
9,gemini-1.5-pro,paper_040,"{\n ""Particle size"": {\n ""SC-NCM83"":...",[each\n\nonly\n\ncase\n\nand\n\nthe\n\nthe\n\n...


In [ ]:
import json

json_paths = [
    "/Users/limseongbeom/Desktop/Code/VOLTAI/data/ground_truth/paper_011_gt.json",
    "/Users/limseongbeom/Desktop/Code/VOLTAI/data/ground_truth/paper_016_gt.json",
    "/Users/limseongbeom/Desktop/Code/VOLTAI/data/ground_truth/paper_022_gt.json",
    "/Users/limseongbeom/Desktop/Code/VOLTAI/data/ground_truth/paper_035_gt.json",
    "/Users/limseongbeom/Desktop/Code/VOLTAI/data/ground_truth/paper_039_gt.json",
    "/Users/limseongbeom/Desktop/Code/VOLTAI/data/ground_truth/paper_040_gt.json",
    "/Users/limseongbeom/Desktop/Code/VOLTAI/data/ground_truth/paper_041_gt.json",
    "/Users/limseongbeom/Desktop/Code/VOLTAI/data/ground_truth/paper_042_gt.json",
    "/Users/limseongbeom/Desktop/Code/VOLTAI/data/ground_truth/paper_044_gt.json",
    "/Users/limseongbeom/Desktop/Code/VOLTAI/data/ground_truth/paper_056_gt.json",
]

# ground_truth 딕셔너리 생성
gt_dict = {}
for path in json_paths:
    paper_num = path.split("/")[-1].split("_")[1]  # 파일명에서 paper_num 추출
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    gt_dict[paper_num] = data["Morphological Properties"]

In [ ]:
df["ground_truth"] = df["paper"].apply(
    lambda x: gt_dict.get(x.replace("paper_", ""), {})
)

In [ ]:
question_template = """  question_text: |
    Fill out in the `null`, `[]` and `{}` values based on the example format:

template: 
    Morphological Properties:
        Particle size: {}
        Particle shape: {}
        Particle distribution: {}
        Coating layer characteristics: {}
        Crystal structure and lattice characteristics: {}question_text: |
    Fill out in the `null`, `[]` and `{}` values based on the example format:

template: 
    Morphological Properties:
        Particle size: {}
        Particle shape: {}
        Particle distribution: {}
        Coating layer characteristics: {}
        Crystal structure and lattice characteristics: {}
        
        """

df["question"] = question_template

In [6]:
df

,model,paper,output,context,ground_truth,question
0,gemini-1.5-pro,paper_016,"{\n ""Particle size"": {\n ""NCM"": ""50 ...",[Figure 2 shows the SEM and TEM images of NCM ...,"{'Particle size': {'NCM': 'very small, sizes r...",question_text: |\n Fill out in the `null`...
1,gemini-1.5-pro,paper_022,"{\n ""Particle size"": {\n ""pristine"":...",[Results and Discussion\nThe XRD patterns of p...,"{'Particle size': {'Pristine': '500 nm', 'V-0....",question_text: |\n Fill out in the `null`...
2,gemini-1.5-pro,paper_041,"{\n ""Particle size"": {\n ""pristine N...","[The SEM images for the pristine NCM-811, g-C3...","{'Particle size': {'NCM-811': '10 to 20 μm', '...",question_text: |\n Fill out in the `null`...
3,gemini-1.5-pro,paper_011,"{\n ""Particle size"": {\n ""NR0"": ""200...",[6\n\nA. Habibi et al. / Journal of Alloys and...,"{'Particle size': {'NR0': '200-300 nm', 'NR1':...",question_text: |\n Fill out in the `null`...
4,gemini-1.5-pro,paper_056,"{\n ""Particle size"": {\n ""pristine N...",[For confirming the microstructural informatio...,"{'Particle size': {'pristine NCM': '11.7 μm', ...",question_text: |\n Fill out in the `null`...
5,gemini-1.5-pro,paper_035,"{\n ""Particle size"": {\n ""S1bare"": ""...","[4. Conclusions\n\nand\n\ncurrent\n\n(GITT),\n...","{'Particle size': {'S1bare': '20 μm', 'S2wash'...",question_text: |\n Fill out in the `null`...
6,gemini-1.5-pro,paper_044,"{\n ""Particle size"": {\n ""NCM622"": n...",[Figure 1a. NCM622 particles were first soaked...,"{'Particle size': {'pristine NCM622': None, 'p...",question_text: |\n Fill out in the `null`...
7,gemini-1.5-pro,paper_039,"{\n ""Particle size"": {\n ""N92"": ""gra...",[2205122 (2 of 13)\n\n© 2022 Wiley-VCH GmbH\n...,"{'Particle size': {'N92': None, 'WN92': None},...",question_text: |\n Fill out in the `null`...
8,gemini-1.5-pro,paper_042,"{\n ""Particle size"": {\n ""N-NCM"": ""1...",[3. Results and discussion\n\nFig. 2 shows the...,"{'Particle size': {'SC-NCM': '2 μm', 'N-NCM': ...",question_text: |\n Fill out in the `null`...
9,gemini-1.5-pro,paper_040,"{\n ""Particle size"": {\n ""SC-NCM83"":...",[each\n\nonly\n\ncase\n\nand\n\nthe\n\nthe\n\n...,"{'Particle size': {'SC-NCM83': '1–4 μm', 'PC-N...",question_text: |\n Fill out in the `null`...


In [ ]:
from datasets import Dataset

# 1️⃣ 모델 리스트
models = ["gemini-1.5-pro", "gemini-2.0-flash", "gpt-4o", "gpt-4o-mini"]

# 2️⃣ 모델별로 DataFrame 분리
model_dfs = {}
for model in models:
    model_df = df[df["model"] == model].copy()
    model_dfs[model] = model_df

# 3️⃣ RAGAS용 열 생성
question_template = """  question_text: |
    Fill out in the `null`, `[]` and `{}` values based on the example format:

template: 
    Morphological Properties:
        Particle size: {}
        Particle shape: {}
        Particle distribution: {}
        Coating layer characteristics: {}
        Crystal structure and lattice characteristics: {}
"""

for model, model_df in model_dfs.items():
    # question 열 추가
    model_df["question"] = [question_template] * len(model_df)

    # output과 context 문자열 변환
    model_df["answer"] = model_df["output"].apply(
        lambda x: json.dumps(x) if isinstance(x, dict) else str(x)
    )

    model_df["ground_truth"] = model_df["output"].apply(
        lambda x: json.dumps(x) if isinstance(x, dict) else str(x)
    )

    # context 처리 (리스트 -> 문자열 리스트)
    model_df["contexts"] = model_df["context"].apply(
        lambda lst: [str(x) if x is not None else "" for x in lst]
    )

    # Dataset 생성
    model_dfs[model] = Dataset.from_pandas(
        model_df[["question", "answer", "ground_truth", "contexts"]]
    )

# 4️⃣ 이제 model_dfs 딕셔너리에 각 모델별 RAGAS Dataset이 존재
# 예: model_dfs["gemini-1.5-pro"]

/Users/limseongbeom/anaconda3/envs/voltai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from dotenv import load_dotenv
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    faithfulness,
    answer_relevancy,
    context_recall,
)

# ==============================================================
# 1️⃣ API KEY 로드
# ==============================================================
load_dotenv()  # .env 파일에서 환경변수 로드

True

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    faithfulness,
    answer_relevancy,
    context_recall,
)

n_runs = 10
multi_results = {model: [] for model in model_dfs.keys()}

for i in range(n_runs):
    for model, dataset in model_dfs.items():
        result = evaluate(
            dataset=dataset,
            metrics=[context_precision, faithfulness, answer_relevancy, context_recall],
            column_map={
                "user_input": "question",
                "ai_output": "answer",
                "reference": "ground_truth",
                "retrieved_contexts": "contexts",
                "response": "answer",
            },
        )
        multi_results[model].append(result)

Evaluating: 100%|██████████| 40/40 [00:51<00:00,  1.30s/it]


In [17]:
multi_results

{'gemini-1.5-pro': [{'context_precision': 0.9516, 'faithfulness': 0.5685, 'answer_relevancy': 0.2955, 'context_recall': 0.5839},
  {'context_precision': 0.9523, 'faithfulness': 0.5735, 'answer_relevancy': 0.2958, 'context_recall': 0.6049},
  {'context_precision': 0.9578, 'faithfulness': 0.5785, 'answer_relevancy': 0.2954, 'context_recall': 0.6283},
  {'context_precision': 0.9464, 'faithfulness': 0.5785, 'answer_relevancy': 0.2956, 'context_recall': 0.6150},
  {'context_precision': 0.9149, 'faithfulness': 0.5462, 'answer_relevancy': 0.2948, 'context_recall': 0.6350},
  {'context_precision': 0.9324, 'faithfulness': 0.5794, 'answer_relevancy': 0.2956, 'context_recall': 0.6550},
  {'context_precision': 0.9260, 'faithfulness': 0.5785, 'answer_relevancy': 0.2957, 'context_recall': 0.5591},
  {'context_precision': 0.9241, 'faithfulness': 0.6535, 'answer_relevancy': 0.2953, 'context_recall': 0.6350},
  {'context_precision': 0.9231, 'faithfulness': 0.5826, 'answer_relevancy': 0.2951, 'context_r

In [24]:
multi_results

{'gemini-1.5-pro': [{'context_precision': 0.9516, 'faithfulness': 0.5685, 'answer_relevancy': 0.2955, 'context_recall': 0.5839},
  {'context_precision': 0.9523, 'faithfulness': 0.5735, 'answer_relevancy': 0.2958, 'context_recall': 0.6049},
  {'context_precision': 0.9578, 'faithfulness': 0.5785, 'answer_relevancy': 0.2954, 'context_recall': 0.6283},
  {'context_precision': 0.9464, 'faithfulness': 0.5785, 'answer_relevancy': 0.2956, 'context_recall': 0.6150},
  {'context_precision': 0.9149, 'faithfulness': 0.5462, 'answer_relevancy': 0.2948, 'context_recall': 0.6350},
  {'context_precision': 0.9324, 'faithfulness': 0.5794, 'answer_relevancy': 0.2956, 'context_recall': 0.6550},
  {'context_precision': 0.9260, 'faithfulness': 0.5785, 'answer_relevancy': 0.2957, 'context_recall': 0.5591},
  {'context_precision': 0.9241, 'faithfulness': 0.6535, 'answer_relevancy': 0.2953, 'context_recall': 0.6350},
  {'context_precision': 0.9231, 'faithfulness': 0.5826, 'answer_relevancy': 0.2951, 'context_r

In [48]:
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    faithfulness,
    answer_relevancy,
    context_recall,
)

# 모델별 평가 결과 저장용 딕셔너리
results = {}

for model, dataset in model_dfs.items():
    result = evaluate(
        dataset=dataset,
        metrics=[context_precision, faithfulness, answer_relevancy, context_recall],
        column_map={
            "user_input": "question",
            "ai_output": "answer",
            "reference": "ground_truth",
            "retrieved_contexts": "contexts",
            "response": "answer",
        },
    )
    results[model] = result

Evaluating: 100%|██████████| 40/40 [00:50<00:00,  1.26s/it]


In [49]:
results

{'gemini-1.5-pro': {'context_precision': 0.9399, 'faithfulness': 0.5785, 'answer_relevancy': 0.2958, 'context_recall': 0.6189},
 'gemini-2.0-flash': {'context_precision': 0.8629, 'faithfulness': 0.7307, 'answer_relevancy': 0.5346, 'context_recall': 0.7459},
 'gpt-4o': {'context_precision': 0.8120, 'faithfulness': 0.7268, 'answer_relevancy': 0.6010, 'context_recall': 0.7514},
 'gpt-4o-mini': {'context_precision': 0.8118, 'faithfulness': 0.6870, 'answer_relevancy': 0.6116, 'context_recall': 0.6553}}